In [1]:
import pandas as pd
import numpy as np
titanic=pd.read_csv('train.csv')

In [20]:
titanic.info

<bound method DataFrame.info of      PassengerId  Survived  Pclass  \
0              1         0       3   
1              2         1       1   
2              3         1       3   
3              4         1       1   
4              5         0       3   
5              6         0       3   
6              7         0       1   
7              8         0       3   
8              9         1       3   
9             10         1       2   
10            11         1       3   
11            12         1       1   
12            13         0       3   
13            14         0       3   
14            15         0       3   
15            16         1       2   
16            17         0       3   
17            18         1       2   
18            19         0       3   
19            20         1       3   
20            21         0       2   
21            22         1       2   
22            23         1       3   
23            24         1       1   
24            25  

In [21]:
titanic.info()   #显示基本信息

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.6+ KB


In [22]:
titanic.describe()  #显示统计信息

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [2]:
from sklearn import svm
titanic['Age']=titanic["Age"].fillna(titanic["Age"].median())
titanic=titanic.drop(['Ticket','Cabin','Name'], axis=1)
titanic.loc[titanic["Sex"] == "male", "Sex"] = 1  #将男性转为1
titanic.loc[titanic["Sex"] == "female", "Sex"] = 0 #将女性转为0
# 首先把所有缺失值替换为"S"
titanic["Embarked"] = titanic["Embarked"].fillna("S")

# 将"S"替换为0，C=1,Q=2
titanic.loc[titanic["Embarked"] == "S", "Embarked"] = 0
titanic.loc[titanic["Embarked"] == "C", "Embarked"] = 1
titanic.loc[titanic["Embarked"] == "Q", "Embarked"] = 2

In [6]:
titanic.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 712 entries, 0 to 890
Data columns (total 10 columns):
PassengerId    712 non-null int64
Survived       712 non-null int64
Pclass         712 non-null int64
Name           712 non-null object
Sex            712 non-null object
Age            712 non-null float64
SibSp          712 non-null int64
Parch          712 non-null int64
Fare           712 non-null float64
Embarked       712 non-null object
dtypes: float64(2), int64(5), object(3)
memory usage: 61.2+ KB


In [ ]:
from patsy import dmatrices
from pandas import DataFrame
from sklearn.cross_validation import KFold
# 定义纳入训练过程的数据列
predictors = ["Pclass", "Sex", "Age", "SibSp", "Parch", "Fare", "Embarked"]
kf = KFold(titanic.shape[0], n_folds=5, random_state=1)
predictions = []
clf = svm.SVC(kernel='poly',gamma=3)
for train, test in kf: 
    # 提取出用作训练的数据行（不含拟合目标）
    train_predictors = (titanic[predictors].iloc[train,:])
    # 提取用于训练的拟合目标
    train_target = titanic["Survived"].iloc[train]
    # 基于训练数据和拟合目标训练模型
    clf.fit(train_predictors, train_target)
    # 接下来在测试集上执行预测
    test_predictions = clf.predict(titanic[predictors].iloc[test,:])
    predictions.append(test_predictions)

F:\python2\python2\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


## 计算误差

In [70]:
predictions = np.concatenate(predictions, axis=0)

# 将浮点数结果映射为二进制结果（0/1表示幸存与否）
#predictions[predictions > .5] = 1
#predictions[predictions <=.5] = 0

# 计算predictions并赋值予accuracy
accuracy=float(len(predictions[predictions==titanic['Survived']]))/float(len(predictions))
accuracy

0.6262626262626263

In [56]:
sum(predictions[predictions==titanic['Survived']])
len(predictions)

891

In [71]:
from sklearn.linear_model import LogisticRegression
from sklearn import cross_validation

# 初始化逻辑回归模型对象
alg = LogisticRegression(random_state=1)

# 计算交叉验证的得分
scores = cross_validation.cross_val_score(alg, titanic[predictors], titanic["Survived"], cv=3)

# 求scores的平均值并输出
print(scores.mean())

0.794612794613
